<a href="https://colab.research.google.com/github/JanetDev/Practica3.3_MD/blob/main/Practica3_3_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import seed
from random import randrange
from math import sqrt
from sklearn.datasets import load_breast_cancer

# Cargar breast cancer dataset de scikit-learn
data = load_breast_cancer()
X, y = data.data, data.target

# Combine características y etiquetas en un conjunto de datos
dataset = [list(X[i]) + [y[i]] for i in range(len(X))]

# Dividir un conjunto de datos en k pliegues
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calcular el porcentaje de precisión
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluar un algoritmo mediante una división de validación cruzada
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted, trees = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores, trees
# Dividir un conjunto de datos según un atributo y un valor de atributo
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calcular el índice de Gini para un conjunto de datos dividido
def gini_index(groups, classes):
	# contar todas las muestras en el punto de división
	n_instances = float(sum([len(group) for group in groups]))
	# Índice de Gini ponderado por suma para cada grupo
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# evitar dividir por cero
		if size == 0:
			continue
		score = 0.0
		# calificar al grupo según el puntaje de cada clase
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# ponderar la puntuación del grupo por su tamaño relativo
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Seleccione el mejor punto de división para un conjunto de datos
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 100, 100, 100, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Crear un valor de nodo terminal
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Crear divisiones secundarias para un nodo o crear una terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# comprobar si no hay división
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# checar la profundidad maxima (max depth)
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

# Construir un árbol de decisiones
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root